In [ ]:
import os, glob
from PIL import Image

import numpy as np
import scipy.interpolate as interp

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import time_to_H

In [ ]:
# Read data

pathFields = "../data/CCGM_500um_375nm_45uN_100Pa_h_1cm_m_93g/muDIC"
pathSnapshots = "../data/CCGM_500um_375nm_45uN_100Pa_h_1cm_m_93g/snapshots"

dt_camera = 0.1

frames_true = np.load(os.path.join(pathFields, "frames.npy")).astype(int)
coords = np.load(os.path.join(pathFields, "coords.npy"))
trueStrain = np.load(os.path.join(pathFields, "true_strain.npy"))

times_true = dt_camera * frames_true
height = time_to_H.time_to_H(times_true)

with open(os.path.join(pathFields, 'params.txt')) as file:
    paramsDIC = file.read()

In [ ]:
# Read calibration

pathCalibration = "../data/Calibrations/Calibration_top_grey/muDIC"

dt_camera_calib = 0.1

frames_true_calib = np.load(os.path.join(pathCalibration, "frames.npy"))
coords_calib = np.load(os.path.join(pathCalibration, "coords.npy"))
trueStrain_calib = np.load(os.path.join(pathCalibration, "true_strain.npy"))

times_true_calib = dt_camera_calib * frames_true_calib
height_calib = time_to_H.time_to_H(times_true_calib)

In [ ]:
# Print shapes

print("From experiment")
print(f"Frames: {frames_true.shape}")
print(f"Coordinates/Strain: {coords[0,0].shape}")

print("")

print("From calibration")
print(f"Frames: {frames_true_calib.shape}")
print(f"Coordinates/Strain: {coords_calib[0,0].shape}")

In [ ]:
# Create interpolator calibration

# Average calibration along y
trueStrain_calib_xx = trueStrain_calib[0,0,0,:,:,:].mean(axis=1)
x0_calib = coords_calib[0,0,:,:,0].mean(axis=1)

# Create interpolator
trueStrain_calib_interp = interp.RegularGridInterpolator((x0_calib, times_true_calib), 
                                                         trueStrain_calib_xx, 
                                                         bounds_error=False, fill_value=None)


In [ ]:
# Prepare snapshots prefix

snapshot_names = glob.glob(pathSnapshots + "/*.tiff")

snapshot_prefix = snapshot_names[0].rsplit("_")
snapshot_prefix = "_".join(snapshot_prefix[:-1])

# At fixed frame

In [ ]:
# Compare contour true_strain without and with calibration

frame_index = -1
frame_true = frames_true[frame_index]
time_true = times_true[frame_index]

print(f'Time: {time_true}s')

x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

trueStrain_toPlot = trueStrain[0,0,0,:,:,frame_index]

# Remove calibration from strain_toPlot
x0_flat = x0.flatten()
trueStrain_visual = trueStrain_calib_interp((x0_flat, np.full_like(x0_flat, time_true)))
trueStrain_visual = trueStrain_visual.reshape(x0.shape)

fig, ax = plt.subplots(figsize=(15,5), ncols=2)

ax[0].set_title(f"True strain raw [frame {frame_true}]")
ax[1].set_title(f"True strain corrected [frame {frame_true}]")

im = ax[0].contourf(x0, y0, trueStrain_toPlot)
fig.colorbar(im, ax=ax[0], label="True strain")

im = ax[1].contourf(x0, y0, trueStrain_toPlot - trueStrain_visual)
fig.colorbar(im, ax=ax[1], label="True strain")

for axi in ax:
    axi.set_xlabel('Position x [pix]')
    axi.set_ylabel('Position y [pix]')

plt.show()

In [ ]:
# Plot the strain along given lines

y_pix_list = [800, 1000, 1200, 1400]

# Find nearest element for each y_pix
y_elem_list = []
for y_pix in y_pix_list:
    y_elem = np.argmin(np.abs(y_pix - y0[y0.shape[0]//2,:]))
    y_elem_list.append(y_elem)

# Plot
fig, ax = plt.subplots(figsize=(10, 10/3.), ncols=2, tight_layout=True)

for y_pix, y_elem in zip(y_pix_list, y_elem_list):
    # Remove calibration from strain_toPlot
    trueStrain_visual = trueStrain_calib_interp((x0[:,y_elem], np.full_like(x0[:,y_elem], time_true)))

    ax[0].plot(x0[:,y_elem], trueStrain_toPlot[:,y_elem], label=f'y_pix = {y_pix}')
    ax[1].plot(x0[:,y_elem], trueStrain_toPlot[:,y_elem] - trueStrain_visual, label=f'y_pix = {y_pix}')

for axi in ax:
    axi.set_xlabel('Horizontal position $X_0$ [pix]')
    axi.legend(loc='lower center', ncol=2)

ax[0].set_ylabel('True strain $\epsilon_t$ [-]')
ax[1].set_ylabel('True strain $\epsilon_t - \epsilon_{t, calib}$ [-]')

ax[0].set_title("True strain")
ax[1].set_title("True strain (corrected)")


plt.show()

# Generate videos

In [ ]:
# Correct the strain

x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

trueStrainXX_correction = np.empty_like(trueStrain[0,0,0])

for i_time in range(times_true.size):

    time = times_true[i_time]
    trueStrainXX_correction[:,:,i_time] = trueStrain_calib_interp((x0.flatten(), np.full(x0.size, time))).reshape(x0.shape)


trueStrainXX_corrected = trueStrain[0,0,0] - trueStrainXX_correction

In [ ]:
# Plot trueStrain corrected only

nbFrames = coords.shape[-1]
frameIndex_start = -1

fig, ax = plt.subplots()

# Set labels
ax.set_title(f"True strain [H = {1e3 * height[frameIndex_start]:.1f}mm]")
ax.set_xlabel('Position $X_0$ [pix]')
ax.set_ylabel('Position $Y_0$ [pix]')

# Set ref coordinates and strain
x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

# Set bounds
xbounds = x0.min(), x0.max()
ybounds = y0.min(), y0.max()
ax.set_xbound(*xbounds)
ax.set_ybound(*ybounds)

# Set static cmap
vmin, vmax = trueStrainXX_corrected.min(), trueStrainXX_corrected.max()
norm = plt.Normalize(vmin, vmax)
cmap = plt.get_cmap('viridis')
n_levels = 100

# Plot strating frame
trueStrain_toPlot = trueStrainXX_corrected[:,:,frameIndex_start]

cont = ax.contourf(x0, y0, trueStrain_toPlot, levels=n_levels, cmap=cmap, norm=norm)
fig.colorbar(cont, ax=ax, norm=norm, cmap=cmap, label="True strain $\epsilon_t$ [-]")


def update(frame_index):
    global cont

    trueStrain_toPlot = trueStrainXX_corrected[:,:,frame_index]
    
    for c in cont.collections:
        c.remove()  # removes only the contours, leaves the rest intact

    cont = plt.contourf(x0, y0, trueStrain_toPlot, levels=n_levels, cmap=cmap, norm=norm)

    # Set limits and title
    ax.set_title(f"True strain [H = {1e3 * height[frame_index]:.1f}mm]")
    ax.set_xbound(*xbounds)
    ax.set_ybound(*ybounds)
    
    return cont

anim = animation.FuncAnimation(fig, update, frames=nbFrames, repeat=False, interval=100)
anim_html = HTML(anim.to_html5_video())

print(f"{nbFrames} frames")


In [ ]:
# Display animation

anim_html

In [ ]:
# Save animation

savepath = os.path.join(pathFields, "animation_trueStrain.html")
with open(savepath, 'w') as f:
    f.write(anim_html.data)

In [ ]:
# Video and DIC on same plot

nbFrames = coords.shape[-1]
frameIndex_start = -1

fig, (axp, ax) = plt.subplots(figsize=(15,5), ncols=2)


# Set labels
ax.set_title(f"True strain [H = {1e3 * height[frameIndex_start]:.1f}mm]")
ax.set_xlabel('Position $X_0$ [pix]')
ax.set_ylabel('Position $Y_0$ [pix]')
axp.set_title(f"Snapshot [H = {1e3 * height[frameIndex_start]:.1f}mm]")
axp.set_xlabel('Position $X_0$ [pix]')
axp.set_ylabel('Position $Y_0$ [pix]')

# Set ref coordinates and strain
x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

# Mask for pictures
xMin, xMax = int(x0.min()), int(x0.max())
yMin, yMax = int(y0.min()), int(y0.max())
X_toKeep = np.arange(xMin, xMax+1).tolist()
Y_toKeep = np.arange(yMin, yMax+1)
maskSnap_XY = np.ix_(Y_toKeep, X_toKeep)

# Set bounds
xbounds = xMin, xMax
ybounds = yMin, yMax
ax.set_xbound(*xbounds)
ax.set_ybound(*ybounds)
#axp.set_xbound(*xbounds)
#axp.set_ybound(*ybounds)

# Set static cmap
vmin, vmax = trueStrainXX_corrected.min(), trueStrainXX_corrected.max()
norm = plt.Normalize(vmin, vmax)
cmap = plt.get_cmap('viridis')
n_levels = 100

# Plot strating frame
trueStrain_toPlot = trueStrainXX_corrected[:,:,frameIndex_start]

cont = ax.contourf(x0, y0, trueStrain_toPlot, levels=n_levels, cmap=cmap, norm=norm)
fig.colorbar(cont, ax=ax, norm=norm, cmap=cmap, label="True strain $\epsilon_t$ [-]")

img = np.array(Image.open(f"{snapshot_prefix}_{frames_true[frameIndex_start]:04d}.tiff"))
print(img.shape)
axp.imshow(img, origin='lower', cmap='Greys_r')

# Plot limits DIC
axp.axvline(xbounds[0], ls=':', c='r')
axp.axvline(xbounds[1], ls=':', c='r')
axp.axhline(ybounds[0], ls=':', c='r')
axp.axhline(ybounds[1], ls=':', c='r')


def update(frame_index):
    global cont

    trueStrain_toPlot = trueStrainXX_corrected[:,:,frame_index]
    
    for c in cont.collections:
        c.remove()  # removes only the contours, leaves the rest intact

    cont = plt.contourf(x0, y0, trueStrain_toPlot, levels=n_levels, cmap=cmap, norm=norm)

    img = np.array(Image.open(f"{snapshot_prefix}_{frames_true[frame_index]:04d}.tiff"))
    im = axp.imshow(img, origin='lower', cmap='Greys_r')

    # Set limits and title
    ax.set_title(f"True strain [H = {1e3 * height[frame_index]:.1f}mm]")
    axp.set_title(f"True strain [H = {1e3 * height[frame_index]:.1f}mm]")
    ax.set_xbound(*xbounds)
    ax.set_ybound(*ybounds)
    #axp.set_xbound(*xbounds)
    #axp.set_ybound(*ybounds)
    
    return cont, im

anim2 = animation.FuncAnimation(fig, update, frames=nbFrames, repeat=False, interval=100)

print(f"{nbFrames} frames")


In [ ]:
anim_html_2 = HTML(anim2.to_html5_video())
anim_html_2